In [7]:
!pip install numpy==1.26.4


In [1]:
import tensorflow as tf
import numpy as np
import cv2
import os
import time


In [2]:
import tensorflow as tf
model_dir = r"C:\Users\Abina\chattojournal\src\TrafficSignDetection\saved_models\efficientdet_d1_coco17_tpu-32\saved_model"
detect_fn = tf.saved_model.load(model_dir)

In [3]:
import tensorflow as tf
classifier_model_path = r"C:\Users\Abina\Downloads\Traffic-Sign-Classification-master\Traffic-Sign-Classification-master\my_model.h5"
classifier = tf.keras.models.load_model(classifier_model_path)

In [11]:
classifier_classes=[
    'Speed limit (20km/h)', 'Speed limit (30km/h)', 'Speed limit (50km/h)',
    'Speed limit (60km/h)', 'Speed limit (70km/h)', 'Speed limit (10km/h)',
    'End of speed limit (80km/h)', 'Speed limit (100km/h)', 'Speed limit (120km/h)',
    'No passing', 'No passing for vehicles over 3.5 metric tons',
    'Right-of-way at the next intersection', 'Priority road', 'Yield', 'Stop',
    'No vehicles', 'Vehicles over 3.5 metric tons prohibited', 'No entry',
    'General caution', 'Dangerous curve to the left', 'Dangerous curve to the right',
    'Double curve', 'Bumpy Road', 'Slippery road', 'Road narrows on the right',
    'Road work', 'Traffic signals', 'Pedestrians', 'Children crossing', 'Bicycles crossing',
    'Beware of ice/snow', 'Wild animals crossing', 'End of all speed and passing limits',
    'Turn right ahead', 'Turn left ahead', 'Ahead only', 'Go straight or right',
    'Go straight or left', 'Keep right', 'Keep left', 'Roundabout mandatory',
    'End of no passing', 'End of no passing by vehicles over 3.5 metric tons'
]

In [12]:
detection_score_threshold = 0.4
classification_score_threshold = 0.3

In [13]:
import os
input_folder = r"C:\Users\Abina\OneDrive\Desktop\roadtrafficsignrecognition\frames\sivaraman"
output_folder = r"C:\Users\Abina\OneDrive\Desktop\roadtrafficsignrecognition\output\sivaraman\video\fullvideo.mp4"
os.makedirs(output_folder, exist_ok=True)


FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\Abina\\OneDrive\\Desktop\\roadtrafficsignrecognition\\output\\sivaraman\\video\\fullvideo.mp4'

In [ ]:
def preprocess_for_classifier(cropped_img):
    # Convert to grayscale if the model expects 1 channel
    gray = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2GRAY)
    
    # Resize to 32x32
    img_resized = cv2.resize(gray, (32, 32))
    
    # Normalize
    img_normalized = img_resized / 255.0
    
    # Expand dimensions to match (batch_size, height, width, channels=1)
    return np.expand_dims(img_normalized[..., np.newaxis], axis=0)


In [ ]:
def run_inference_on_image(image):
    image_np_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  #convert to RGB
    input_tensor = tf.convert_to_tensor(image_np_rgb)   #convert to tensor
    input_tensor = input_tensor[tf.newaxis, ...]
    detections = detect_fn(input_tensor)  #then detect
    return detections  #returns the array for detections

In [ ]:
import cv2
import time
import os

# Video writer settings
output_video_path = r"C:\Users\Abina\OneDrive\Desktop\roadtrafficsignrecognition\output\sivaraman\video"

# Get size from first frame for video writer (assuming all frames same size)
first_image_path = os.path.join(input_folder, os.listdir(input_folder)[0])
first_image = cv2.imread(first_image_path)
if first_image is None:
    raise RuntimeError("Could not read the first image for video size setup")

height, width, _ = first_image.shape

# Define codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or use 'XVID', 'MJPG', etc.
fps = 10  # set frame rate as you like
video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

total_start_time = time.time()
frame_count = 0

for img_name in os.listdir(input_folder):
    frame_start_time = time.time()

    img_path = os.path.join(input_folder, img_name)
    image = cv2.imread(img_path)
    if image is None:
        print(f"Warning: Could not read {img_name}")
        continue

    height, width, _ = image.shape
    detections = run_inference_on_image(image)

    boxes = detections['detection_boxes'].numpy()[0]
    scores = detections['detection_scores'].numpy()[0]

    for box, score in zip(boxes, scores):
        if score < detection_score_threshold:
            continue

        ymin, xmin, ymax, xmax = box
        left = int(xmin * width)
        right = int(xmax * width)
        top = int(ymin * height)
        bottom = int(ymax * height)

        cropped_img = image[top:bottom, left:right]
        if cropped_img.size == 0:
            continue

        input_for_cls = preprocess_for_classifier(cropped_img)
        preds = classifier.predict(input_for_cls)
        class_id = np.argmax(preds)
        cls_score = preds[0][class_id]

        if cls_score < classification_score_threshold:
            label_text = "Unknown"
        else:
            label_text = f"{classifier_classes[class_id]} ({cls_score:.2f})"

        cv2.rectangle(image, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.putText(image, label_text, (left, top - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    save_path = os.path.join(output_folder, img_name)
    cv2.imwrite(save_path, image)

    # Write frame to video
    video_writer.write(image)

    print(f"Processed and saved: {save_path}")
    frame_end_time = time.time()
    frame_duration = frame_end_time - frame_start_time
    print(f"Processed {img_name} in {frame_duration:.3f} seconds")
    frame_count += 1

# Release the video writer after all frames are processed
video_writer.release()

total_end_time = time.time()
total_duration = total_end_time - total_start_time
print(f"Processed {frame_count} frames in {total_duration:.3f} seconds")
print(f"Average time per frame: {total_duration/frame_count:.3f} seconds")
print(f"Video saved at: {output_video_path}")
